<a href="https://colab.research.google.com/github/OmarMohammed88/heart_disease/blob/main/models/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install -U gdown

  Installing build dependencies ... canceled
ERROR: Operation cancelled by user


In [16]:
#download folder from this link
!gdown --id '1--u4JDjwqNvGNebKG_PG-qzKJQV83gPd' 
!gdown --id '1cNv8-m22M_I-XYeIe0GHWnqGp1x32BfR' 
!gdown --id '1wc0_0Ydu9UEj-8T2r8PfT1fbXU-otZcN' 
!gdown --id '1VA2x20iKDo_p079HL1G-K8h5OFb04FaV' 

Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/gdown/cli.py", line 105, in main
    use_cookies=not args.no_cookies,
  File "/usr/local/lib/python3.7/dist-packages/gdown/download.py", line 110, in download
    res = sess.get(url, stream=True)
  File "/usr/local/lib/python3.7/dist-packages/requests/sessions.py", line 542, in get
    return self.request('GET', url, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/requests/sessions.py", line 515, in request
    prep = self.prepare_request(req)
  File "/usr/local/lib/python3.7/dist-packages/requests/sessions.py", line 453, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/usr/local/lib/python3.7/dist-packages/requests/models.py", line 318, in prepare
    self.prepare_url(url, params)
  File "/usr/local/lib/python3.7/dist-packages/requests/models.py", line 392, in prepare_url
    raise MissingSchema(

In [ ]:
!pip install git+https://github.com/flairNLP/flair.git

!pip install sentence-transformers

In [76]:
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings, BertEmbeddings, CharacterEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from typing import List
import pandas as pd
import os
import random
import xml.etree.cElementTree as ET
from sklearn.preprocessing import LabelEncoder
import nltk.data
from nltk import sent_tokenize
# from flair.data_fetcher import NLPTaskDataFetcher
from pathlib import Path
from flair.data import Corpus
from flair.datasets import ClassificationCorpus
from flair.embeddings import SentenceTransformerDocumentEmbeddings

In [77]:
data_folder = Path('/content/')

# load corpus containing training, test, and dev data
corpus= ClassificationCorpus(data_folder,
                                        test_file='test_flair.txt',
                                          dev_file='dev_flair.txt',
                                          train_file='train_flair.txt',
                                        label_type='class')

2022-04-14 22:27:33,086 Reading data from /content
2022-04-14 22:27:33,088 Train: /content/train_flair.txt
2022-04-14 22:27:33,091 Dev: /content/dev_flair.txt
2022-04-14 22:27:33,092 Test: /content/test_flair.txt
2022-04-14 22:27:33,890 Initialized corpus /content (label type name is 'class')


In [78]:
label_dict = corpus.make_label_dictionary(label_type='class')


2022-04-14 22:27:34,974 Computing label dictionary. Progress:


59073it [00:11, 5186.47it/s]

2022-04-14 22:27:46,369 Dictionary created for label 'class' with 93 values: Other (seen 52799 times), hypertension.mention.during_dct (seen 378 times), smoker.unknown (seen 299 times), hypertension.high_bp.during_dct (seen 297 times), diabetes.mention.before_dct (seen 262 times), diabetes.mention.during_dct (seen 247 times), cad.event.before_dct (seen 221 times), medication.statin..before_dct (seen 210 times), medication.aspirin..after_dct (seen 209 times), medication.beta_blocker..before_dct (seen 186 times), cad.mention.after_dct (seen 170 times), medication.aspirin..before_dct (seen 165 times), medication.ace_inhibitor..after_dct (seen 158 times), diabetes.mention.after_dct (seen 157 times), medication.beta_blocker..during_dct (seen 156 times), smoker.never (seen 154 times), hyperlipidemia.mention.after_dct (seen 136 times), hypertension.mention.after_dct (seen 135 times), smoker.past (seen 133 times), medication.beta_blocker..after_dct (seen 131 times)


In [79]:
embedding_types = [
     BertEmbeddings(),
    CharacterEmbeddings(),
]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a Be

In [80]:
document_embeddings = DocumentRNNEmbeddings(embedding_types,
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

In [81]:
from flair.models import TextClassifier

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict,label_type='class', multi_label=False)

In [82]:
classifier = classifier.load("/content/best-model.pt")

2022-04-14 22:27:56,345 loading file /content/best-model.pt


In [83]:
trainer = ModelTrainer(classifier, corpus)

In [84]:
final_test = trainer.final_test("/content/",32,("micro avg", "f1-score"))

2022-04-14 22:27:57,227 ----------------------------------------------------------------------------------------------------
2022-04-14 22:27:57,234 loading file /content/best-model.pt


100%|██████████| 1343/1343 [03:04<00:00,  7.28it/s]


2022-04-14 22:31:02,667 Evaluating as a multi-label problem: False
2022-04-14 22:31:02,968 0.9366	0.9366	0.9366	0.9366
2022-04-14 22:31:02,970 
Results:
- F-score (micro) 0.9366
- F-score (macro) 0.2899
- Accuracy 0.9366

By class:
                                                precision    recall  f1-score   support

                                         Other     0.9752    0.9881    0.9816     38375
               hypertension.mention.during_dct     0.7778    0.8558    0.8149       319
                   diabetes.mention.during_dct     0.5261    0.5904    0.5564       188
                   diabetes.mention.before_dct     0.5202    0.5124    0.5163       201
               hypertension.high_bp.during_dct     0.8441    0.8920    0.8674       176
                 medication.statin..before_dct     0.6222    0.7417    0.6767       151
                 medication.aspirin..after_dct     0.7013    0.6750    0.6879       160
                          cad.event.before_dct     0.5310    0.

**Classification Report after Normalizing the Classes**



In [94]:
from sklearn.metrics import confusion_matrix, classification_report
import csv

def normalize_function(text):
  try:
    text = text.split(".")[0]
    return text
  except:
    return text

def normalizing_classification_report(path):
  all_text = []
# open .tsv file
  with open(path) as file:

      tsv_file = csv.reader(file, delimiter="\t")     
      for line in tsv_file:
          all_text.append(line)
  #reformat the test file for normalizing process
  no_nan_li = [i for i in all_text if len(i)!=0]
  df = pd.DataFrame()
  df['text'] = [i[0].strip() for i in no_nan_li]
  truth = df[df.text.str.contains("- Gold:")]
  predcs = df[df.text.str.contains("- Pred:")]
  truth_no_normalize = [i.split(":")[-1] for i in truth.text.tolist()]
  preds_no_normalize = [i.split(":")[-1] for i in predcs.text.tolist()]
  normalize_truth = [normalize_function(text) for text in truth_no_normalize]
  normalize_preds = [normalize_function(text) for text in preds_no_normalize]
  class_name = list(set(normalize_truth))
  print(classification_report(normalize_truth,normalize_preds,target_names=class_name))

In [95]:
normalizing_classification_report("test.tsv")

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

       diabetes       0.98      0.99      0.98     38375
         smoker       0.70      0.60      0.65       446
          Other       0.79      0.69      0.74       582
          obese       0.00      0.00      0.00        13
            cad       0.87      0.56      0.68       231
    family_hist       0.87      0.90      0.88       664
   hypertension       0.92      0.85      0.88      2062
 hyperlipidemia       0.82      0.92      0.87       116
     medication       0.82      0.51      0.63       457

       accuracy                           0.96     42946
      macro avg       0.75      0.67      0.70     42946
   weighted avg       0.96      0.96      0.96     42946



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
